### Do the imports
- If I am doing any processing of files in a folder I like to import os and glob
- import cellpose models for training of inferance
- import something to read your images 
- import libraries to save your results


In [1]:
import os
import glob
import sys

import tifffile # read an write tiff files
from cellpose import models
import napari
import torch

import numpy as np
from skimage.filters import gaussian

from skimage.transform import rescale
import skimage as ski

import utils

Create the napari viewer to visualize images and mask

In [2]:
viewer = napari.Viewer()

Find the image files to work with. Different path for Window, Mac, and Linux

In [3]:
cells = ski.io.imread('Data/120_img.png')
cells.shape

(384, 512, 3)

In [6]:
cells[..., 2].max()

0

In [7]:
nuclei = cells[..., 0]
cyto = cells[..., 1]

Add the image to the napari viewer

In [5]:
viewer.layers.clear()

viewer.add_image(cyto, colormap='green', blending='additive')
viewer.add_image(nuclei, colormap='magenta', blending='additive')



<Image layer 'nuclei' at 0x2f0f598a0>

In [8]:
if sys.platform == 'darwin': 
    d = torch.device('mps')
    nuclei_model = models.Cellpose(gpu=False, device=d, model_type='nuclei')
    cyto_model = models.Cellpose(gpu=False, device=d, model_type='cyto2')
else:   
    nuclei_model = models.Cellpose(gpu=True, model_type='nuclei')
    cyto_model = models.Cellpose(gpu=True, model_type='cyto2')


The nuclei image is very hazy - do a background subtration using white tophat.  

In [9]:
bs_nuc = ski.morphology.white_tophat(nuclei, ski.morphology.disk(10))

In [10]:
viewer.add_image(bs_nuc)

<Image layer 'bs_nuc' at 0x2f6e631f0>

Use nuclei model to segment the nuclei image.

In [13]:

masks, flows, styles, diams = nuclei_model.eval(bs_nuc, channels=[0, 0],   #[0, 3] might work best
                                         diameter=15,
                                         cellprob_threshold=-1,
                                         flow_threshold=1)

In [14]:
viewer.layers.clear()
viewer.add_image(nuclei, colormap='magenta', blending='additive')
viewer.add_image(cyto, colormap='green', blending='additive')
viewer.add_image(bs_nuc, colormap='cyan', blending='additive', visible=False)
viewer.add_labels(masks)

<Labels layer 'masks' at 0x2f99d3f40>

In [ ]:
viewer.add_image(flows[0], blending="additive")

Use the cyto model to segment the cytoplasm with default diameter

In [15]:
cyto_masks, flows, styles, diams = cyto_model.eval(cyto, channels=[0, 0],
                                         diameter=30)
viewer.add_labels(cyto_masks)

<Labels layer 'cyto_masks' at 0x340e2ff70>

Can you find anything that looks questionable?

The cellpose cyto models can use a second channel with nuclei to help find and serparate cells.

In the `channels` parameter, make the first index the channel to segment, the second index the channel with nuclei. It is important to start these at 1, not zero. 

In [16]:
data = np.stack([cyto, bs_nuc])
data_masks, flows, styles, diams = cyto_model.eval(data, channels=[1, 2],
                                         diameter=30)

viewer.add_labels(data_masks)

<Labels layer 'data_masks' at 0x340ee3eb0>

Compare the results of the different methods. It might be easier if the masks are set to contours.

In [17]:
gt = ski.io.imread("Data/120_masks.png")
viewer.add_labels(gt)

<Labels layer 'gt' at 0x3545d44f0>